# Continuous Bag of Words (Implementation)

In this notebook, I implement the Continuous Bag of Words algorithm, which is a word2vec algorithm for calculating word embeddings. First, we must start off with some actual words data to train the CBOW model on. For this, I will use Text8, a text file containing the first 100 MB of cleaned text from Wikipedia. 

## Motivation

One-hot encoding doesn't carry much meaning - word embeddings allow us to capture relationship between words.

In CBOW, we predict a target word based on context words.

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.corpus import gutenberg
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import json
import os
import torch
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1. Prepare data

CBOW works by training a model to predict the target word based off of on the context word, allowing us to use the trained parameters as word embeddings. Thus, we must first prepare the words to be inputted into the neural network, so we need to one-hot encode. This entails a few main steps:

1. Prepare Corpus: We will use an excerpt from "Alice in Wonderland" by Lewis Carroll
2. One-hot encode each word in the vocabulary, resulting in a dictionary mapping from each word to a sparse vector.
2. Find the context for each word in the vocabulary, resulting in a dictionary mapping from each word to its context words.
3. One-hot encode the word-context dictionary using the word-onehot dictionary, resulting in a one-hot encoded word-context dictionary.

### 1.1 Prepare Corpus

In [4]:
corpus_path = 'corpus.txt'
with open(corpus_path, 'r', encoding='utf-8') as f:
    corpus = f.read()

print(corpus)

# Load the first 1000 words from Emma by Jane Austen to use as a corpus
# nltk.download('gutenberg')
# corpus = gutenberg.words('austen-emma.txt')[:1162]
# corpus = [word.lower() for word in corpus if word.isalpha()]
# print("Number of words in corpus:", len(corpus))
# 
# print("First 200 words of corpus:")
# print(corpus[:200])

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.

There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, 'Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and t

In [6]:
# Preprocess words (lowercase, remove punctuation, tokenization)

# Sample preprocessed text
corpus = corpus.lower()

# Remove punctuation
translator = str.maketrans('', '', string.punctuation)
corpus = corpus.translate(translator)

# Tokenize the text
words = word_tokenize(corpus)

# Create vocabulary
vocabulary = set(words)

print("Vocabulary:", vocabulary)
print("Number of unique words:", len(vocabulary))

Vocabulary: {'sleepy', 'was', 'either', 'sister', 'time', 'natural', 'pop', 'very', 'at', 'fortunately', 'eyes', 'rabbit', 'ran', 'pink', 'bank', 'down', 'would', 'for', 'much', 'across', 'way', 'hot', 'no', 'rabbithole', 'after', 'peeped', 'itself', 'getting', 'dear', 'then', 'sitting', 'first', 'actually', 'of', 'tired', 'daisies', 'the', 'seen', 'nor', 'beginning', 'worth', 'started', 'to', 'day', 'trouble', 'feet', 'feel', 'late', 'nothing', 'having', 'conversation', 'over', 'ought', 'see', 'oh', 'mind', 'hedge', 'did', 'field', 'is', 'well', 'a', 'could', 'conversations', 'use', 'be', 'there', 'this', 'white', 'chain', 'considering', 'occurred', 'think', 'under', 'looked', 'hear', 'that', 'twice', 'and', 'into', 'pleasure', 'once', 'her', 'say', 'burning', 'reading', 'quite', 'large', 'ever', 'when', 'on', 'made', 'suddenly', 'waistcoatpocket', 'own', 'thought', 'it', 'up', 'get', 'by', 'picking', 'or', 'what', 'hurried', 'had', 'pictures', 'all', 'have', 'in', 'with', 'alice', 'i

In [ ]:
window_size = 2
def build_word_context_pairs(corpus=corpus, vocabulary=vocabulary, window_size=window_size):
    word_context_pairs = {}
    
    for target_word in vocabulary:
        for word in corpus:
            if target_word == word and word.index() - window_size >= 0 and word.index() + window_size <= len(corpus) - 1:
                word_context_pairs[target_word] = [word for word in corpus if word.index() >= word.index() - window_size and word.index() <= word.index() + window_size]
                
                
                

In [6]:
window_size = 3
    
def build_context_word_pairs(corpus, window_size=window_size, vocab_size=vocab_size):
    """   
    Args:
        words (list of str):List containing all the words (vocabulary)
        window_size (int): The number of words to consider on each side of the target word.

    Returns:
        dict: Dictionary where keys are words and values are their corresponding contexts.
    """
    context_length = window_size*2
    words_to_context = {} 
    for index, word in enumerate(words):
        start_index = max(0, index - window_size)
        end_index = min(len(words), index + window_size + 1)       
        word_context = words[start_index:index] + words[index + 1:end_index]
        
        if word in words_to_context:
            words_to_context[word].append(word_context)
        else:
            words_to_context[word] = [word_context]
            
    return words_to_context


### 1.3 Retrieve Word Contexts

(Since in CBOW we are trying to predict the center word based on context words, we first need to define which context words correlate to each word. )
In addition to encoding each word, we must also retrieve the contexts of each word. We do this by finding the surrounding words of the target word in different sentences, defined by some window size. Let's choose a window size of 4.

The below code creates a dictionary where each key is a word and its corresponding object is the context words.

### 1.2 One Hot Encode Words
Now that we have prepared a list of words we will use as our vocabulary to generate word embeddings on, let's create a dictionary to one hot encode each of these words. This is so can we feed the words into our model so it can train parameters.

In [28]:
encoder = OneHotEncoder() 
vocab_array = np.array(vocab).reshape(-1, 1)
encoder.fit(vocab_array)
words_to_one_hot = {word: encoder.transform([[word]])[0] for word in vocab} 

<class 'dict'>


Let's save the words_to_one_hot dictionary to a json file.

In [29]:
words_to_one_hot_list = {word: one_hot.toarray().tolist()[0] for word, one_hot in words_to_one_hot.items()} # Convert csr_matrix to list before saving to JSON

file_path = 'data/words_to_one_hot.json'
with open(file_path, 'w') as file:
    json.dump(words_to_one_hot_list, file, indent=4)

In [ ]:
words_to_context = build_context_dictionary(vocab)

As we did for the words_to_one_hot dictionary, let's save the words_to_context dictionary as a JSON file as well.

In [ ]:
file_path = 'data/words_to_context.json'

with open(file_path, 'w') as file:
    json.dump(words_to_context, file, indent=4)

The code below uses the words_to_context JSON file to load the words_to_context dictionary (assuming you have saved the words_to_context JSON file previously)

In [31]:
if os.path.exists(file_path):
        with open(file_path) as file:
            words_to_context = json.load(file)

### 1.4 Encode Words-Context Dictionary

Now we have two dictionaries:
- `words_to_context`: A dictionary where each key is a word and the corresponding value is a list of context words.
- `words_to_one_hot`: A dictionary mapping each word to its one-hot encoded vector.

Let's use these two dictionaries to finally create a dictionary where both keys and their corresponding context words are one-hot vectors. 

In [46]:
def convert_words_to_one_hot(words_to_context, words_to_one_hot):
    """
    Convert a dictionary of words and their contexts from words to one-hot encoded vectors.

    Args:
        words_to_context (dict): A dictionary where keys are words and values are lists of context words.
        word_to_one_hot (dict): A dictionary mapping words to their corresponding one-hot encoded vectors.

    Returns:
        dict: A dictionary where both keys and their corresponding context words are converted to string representations of one-hot vectors.
    """
    one_hot_dict = {}

    for word, contexts in words_to_context.items():
        if word in words_to_one_hot:
            # Convert the one-hot vector for the key word to a string key
            one_hot_word = json.dumps(words_to_one_hot[word].toarray()[0].tolist())
            # Convert the one-hot vectors for all context words to string keys
            one_hot_contexts = [json.dumps(words_to_one_hot[ctx].toarray()[0].tolist()) for ctx in contexts if ctx in words_to_one_hot]
            # Store in the new dictionary using the string representation of the one-hot vector as the key
            one_hot_dict[one_hot_word] = one_hot_contexts

    return one_hot_dict

encoded_words_to_context = convert_words_to_one_hot(words_to_context, words_to_one_hot)

NameError: name 'word_to_one_hot' is not defined

As we did for the other dictionaries, let's save this as a JSON file.

In [38]:
file_path = 'data/encoded_words_to_context.json'
with open(file_path, 'w') as file:
    json.dump(encoded_words_to_context, file, indent=4)

### 1.5 Create Usable Dataset 

Now that we have a dictionary containing the words and their corresponding contexts represented as one-hot vectors, `encoded_words_to_context`, we can use this to finally create a usable dataset to train the model. Since in CBOW we are trying to predict the target word based off of the context words, the features here will be the context words, and the "label" will be the target word.

First, we must prepare the data by extracting features and labels from the dictionary appropriately.

In [44]:
inputs = []
targets = []

for target, contexts in encoded_words_to_context.items():
    # Assuming contexts is a list of one-hot encoded vectors
    context_sum = sum(torch.tensor(context) for context in contexts)  # Summing context vectors
    avg_context = context_sum / len(contexts)  # Averaging
    inputs.append(avg_context)
    targets.append(torch.tensor(target))  # Target vector

inputs = torch.stack(inputs)  # Convert list of tensors to a tensor
targets = torch.stack(targets)

ZeroDivisionError: division by zero

In [ ]:
# Convert inputs and targets into train and test sets
X_train, X_test, y_train, y_test = train_test_split(inputs.numpy(), targets.numpy(), test_size=0.2, random_state=42)


## 2. Define and Train Model

In [45]:
first_five_keys = list(encoded_words_to_context.keys())[:5]

# Print the keys and their corresponding values
for key in first_five_keys:
    print(f"Key: {key}")
    print("Values:")
    if encoded_words_to_context[key]:  # Check if there are any values
        for value in encoded_words_to_context[key]:
            print(value)
    else:
        print("This key has no values or an empty list.")
    print("\n")  # Print a newline for better separation between entries

Key: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,